In [ ]:
!pip install  peft accelerate transformers flask pyngrok bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-man

In [ ]:
!pip install flask-cors

In [ ]:
from pyngrok import ngrok

# Authenticate ngrok with your token
ngrok.set_auth_token("Your_NGROK_TOKEN")

import torch
from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from flask_cors import CORS
from peft import PeftModel, PeftConfig

model_name = "meta-llama/Meta-Llama-3-8B"
app =  Flask(__name__)
CORS(app)
# Define quantization configuration
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

# Load model with quantization
b_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)

model = PeftModel.from_pretrained(b_model, "abhi317/results_007")

model.generation_config.cache_implementation="static"

compiled_model = torch.compile(model,mode="reduce-overhead",fullgraph=True)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [ ]:
model.generation_config.cache_implementation="static"

compiled_model = torch.compile(model,mode="reduce-overhead",fullgraph=True)

In [ ]:
ngrok.set_auth_token("2hDfFyI883tphFXw4JP6rLgAdfI_61agTWRhLtfSwJa3i7R3E")

In [ ]:
def answer_question(question, model, tokenizer):
    tokenizer.pad_token = tokenizer.eos_token
    input_ids = tokenizer(question,return_tensors="pt",truncation=True).to("cuda")
    attention_mask = input_ids['attention_mask'].to("cuda")
    outputs = model.generate(
    input_ids['input_ids'],
    max_length=200,
    num_beams=1,
    attention_mask=attention_mask,
    temperature=0.7,
    repetition_penalty=1.2,
    top_p=0.95,
    top_k=50,
    do_sample=True,
    use_cache=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

@app.route('/predict', methods=['POST'])
def predict15():
    data = request.get_json()
    print(data)
    question = data['text']
    system_prompt = "Act as a chatbot and answer questions in the field of machine learnig. Explain machine learning concepts in a clear, concise, and easy-to-understand way, avoiding technical jargon whenever possible. Use examples and analogies to illustrate complex ideas."
    questions = system_prompt+question
    answer = answer_question(question, compiled_model, tokenizer)
    return jsonify({'prediction': answer})

if __name__ == '__main__':
    # Start ngrok
    public_url = ngrok.connect(5000)
    print(f' * ngrok tunnel "{public_url}" -> "http://127.0.0.1:5000"')

    # Run Flask app
    app.run()

 * ngrok tunnel "NgrokTunnel: "https://5726-104-198-108-43.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [05/Jun/2024 10:15:25] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Jun/2024 10:15:26] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Jun/2024 10:15:45] "GET /predict HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [05/Jun/2024 10:17:23] "OPTIONS / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Jun/2024 10:17:41] "OPTIONS /predict HTTP/1.1" 200 -
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'text': 'What is ML'}


/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [05/Jun/2024 10:18:18] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Jun/2024 10:18:47] "OPTIONS /predict HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'text': 'What is MAchine Learning?'}


INFO:werkzeug:127.0.0.1 - - [05/Jun/2024 10:19:11] "POST /predict HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'text': 'Tell me about ML.'}


INFO:werkzeug:127.0.0.1 - - [05/Jun/2024 10:21:16] "POST /predict HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'text': 'what is ML.'}


INFO:werkzeug:127.0.0.1 - - [05/Jun/2024 10:22:54] "POST /predict HTTP/1.1" 200 -
